In [1]:
from bs4 import BeautifulSoup
import requests
import time
import os
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
path="C:\Program Files (x86)\msedgedriver.exe"

<>:11: SyntaxWarning: invalid escape sequence '\P'
<>:11: SyntaxWarning: invalid escape sequence '\P'
C:\Users\9616k\AppData\Local\Temp\ipykernel_22932\3344381657.py:11: SyntaxWarning: invalid escape sequence '\P'
  path="C:\Program Files (x86)\msedgedriver.exe"


In [2]:
BASE_URL="https://xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c"

In [3]:
import re

def clean_name(folder_name):
    # Replace special characters in folder name with '-'
    # For special cases like while scraping data for category: Guidelines for Release/Notification, Provisional Notification and De-notification of Cultivars
    cleaned_name = re.sub(r'[\/:*?"<>| ]', '-', folder_name)
    cleaned_name = cleaned_name.replace("\"", "")
    cleaned_name = cleaned_name.replace("'", "")
    return cleaned_name


In [13]:
def fetchData(soup):
    x=0
    MiddleColumn = soup.find('div', id='MiddleColumn')
    if MiddleColumn:
        # Remove all tags and add only text
        text_content = ''
        for element in MiddleColumn.contents:
            x+=1
            if element.name == 'h3' or element.name == 'h4' or element.name == 'p':
                text_content += str(element.text) + '\n'
            else:
                text_content += str(element)
    if x<3:
        MiddleColumn = soup.find('div', id='MiddleColumn_internal')
        # Remove all <a> tags from MiddleColumn
        for a_tag in MiddleColumn.find_all('a'):
            a_tag.decompose()
        # Remove all tags and add only text
        text_content = ''
        for element in MiddleColumn.contents:
            if element.name == 'h3' or element.name == 'h4' or element.name == 'p':
                text_content += str(element.text) + '\n'
            else:
                text_content += str(element)
    # Remove HTML tags using regular expression
    return re.sub(r'<.*?>', '', text_content)

In [5]:
def webScrapeSubCategory(url, folder_path='.'):
    #For Delay
    try:
        data = requests.get(url, verify=False) #Ignoring SSL certificate verification
        soup = BeautifulSoup(data.text, "html.parser") # Generating Soup
#         print(soup)
        a_tags = soup.find_all('a', class_='folderfile_name') #Fetching a tags
        result = []
        if a_tags:
            for tag in a_tags:
                category_name = tag.text.strip() #removing extra spaces or lines
                category_folder_name = clean_name(category_name) #clean name
                category_folder_path = os.path.join(folder_path, category_folder_name) # calulate path
                os.makedirs(category_folder_path, exist_ok=True) # If directory does not exitst create one
                print(f"Scraping data for category: {category_name} {url}") #Show message --> For Debugging
                category_url = BASE_URL + tag.get('href') #Next website
                subcategories = webScrapeSubCategory(category_url, category_folder_path) # recursively call
                result.append({category_name: subcategories})
        else:
            print(f"Fetching content for {url}")
            driver = webdriver.Edge() 
            driver.get(url)
            time.sleep(5)
            html_content = driver.page_source
            driver.quit() 
            soup = BeautifulSoup(html_content, "html.parser") # Generating Soup
            title = soup.find('h3', class_='title')
            if title is None:
                title = soup.find('h3')
            scrapped_data = title.text + "\n\n" + fetchData(soup)     
            with open(os.path.join(folder_path, clean_name(title.text)+'.txt'), 'w', encoding='utf-8') as file:
                file.write(scrapped_data)
            return scrapped_data
        return result
    except Exception as e:
        print("EXCEPTION OCCURRED",e)
        return ""


In [14]:
def webScrape(url, start,end,folder_path='.'):
    #For Delay
    try:
        data = requests.get(url, verify=False) #Ignoring SSL certificate verification
        soup = BeautifulSoup(data.text, "html.parser") # Generating Soup
        a_tags = soup.find_all('a', class_='folderfile_name') #Fetching a tags
        result = []
        if a_tags:
            for tag in a_tags[start:end+1]:
                category_name = tag.text.strip() #removing extra spaces or lines
                category_folder_name = clean_name(category_name) #clean name
                category_folder_path = os.path.join(folder_path, category_folder_name) # calulate path
                os.makedirs(category_folder_path, exist_ok=True) # If directory does not exitst create one
                print(f"Scraping data for category: {category_name} {url}") #Show message --> For Debugging
                category_url = BASE_URL + tag.get('href') #Next website
                subcategories = webScrapeSubCategory(category_url, category_folder_path) # recursively call
                result.append({category_name: subcategories})
        else:
            title = soup.find('h3', class_='card-title title')
            scrapped_data = title.text + "\n\n" + fetchData(url)
            with open(os.path.join(folder_path, clean_name(title.text)+'.txt'), 'w', encoding='utf-8') as file:
                file.write(scrapped_data)
            return scrapped_data
    
        return result
    except Exception as e:
        print("EXCEPTION OCCURRED",e)


In [15]:
#For generating list with indices
data = requests.get(BASE_URL+"/e-governance", verify=False) #Ignoring SSL certificate verification
soup = BeautifulSoup(data.text, "html.parser") # Generating Soup
a_tags = soup.find_all('a', class_='folderfile_name') #Fetching a tags
# print(soup)
result = []
if a_tags:
    for i,tag in enumerate(a_tags):
        result.append([tag.text,i])
    print(result)
else:
    print("No a tags found")

[['ई-शासन', 0], ['ई-शासन ऑनलाईन सेवाएं', 1], ['उत्कृष्ट प्रयास', 2], ['एफ.एस.एस.ए.आई(FSSAI) लाइसेंस की आवश्यकता एवं प्रक्रिया', 3], ['डिजिटल इंडिया', 4], ['डिजिटल भुगतान ', 5], ['नागरिक सेवाएं', 6], ['भारत में ई-शासन', 7], ['भारत में विधिक सेवाएँ', 8], ['महिलाएं एवं ई-शासन ', 9], ['मोबाइल शासन', 10], ['राज्य', 11], ['वीएलई के लिए संसाधन', 12], ['वीएलई समाचार ', 13], ['सूचना का अधिकार ', 14]]


C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [20]:
allSectors=webScrape(BASE_URL+"/e-governance",0,14,"VIKASPEDIA_DATA_E_SHASAN") #START AND END ARE BOTH INCLUDING INDICES

C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Scraping data for category: वीएलई समाचार https://xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c/e-governance


C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Scraping data for category: अपना धन के लिए रजिस्टर करें https://xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c/e-governance/93594090f932908-93892e93e91a93e930


C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Fetching content for https://xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c/e-governance/about-rti-act-2005/93894291a92893e-924915928940915/93894291a92893e-92a94d93094c92694d92f94b91793f915940-90592793f92893f92f92e-2000/साइबर-अपील-अधिकरण
Scraping data for category: सुरक्षित इलेक्ट्रानिक अभिलेख, चिह्नक और प्रमाणकर्ता प्राधिकारियों का विनियमन https://xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c/e-governance/about-rti-act-2005/93894291a92893e-924915928940915/93894291a92893e-92a94d93094c92694d92f94b91793f915940-90592793f92893f92f92e-2000


C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching content for https://xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c/e-governance/about-rti-act-2005/93894291a92893e-924915928940915/93894291a92893e-92a94d93094c92694d92f94b91793f915940-90592793f92893f92f92e-2000/93894193091594d93793f924-90793294791594d91f94d93093e92893f915-90592d93f932947916-91a93f93994d928915-935-93894193091594d93793f924-90793294791594d91f94d93093e92893f915-90592d93f932947916-914930-92a94d93092e93e92391593094d92493e
EXCEPTION OCCURRED [Errno 2] No such file or directory: 'VIKASPEDIA_DATA_E_SHASAN\\सूचना-का-अधिकार\\सूचना-तकनीक\\सूचना-प्रौद्योगिकी-अधिनियम,-2000\\सुरक्षित-इलेक्ट्रानिक-अभिलेख,-चिह्नक-और-प्रमाणकर्ता-प्राधिकारियों-का-विनियमन\\सुरक्षित-इलेक्ट्रानिक-अभिलेख,-चिह्नक-और-प्रमाणकर्ता-प्राधिकारियों-का-विनियमन.txt'
Scraping data for category: सूचना प्रौद्योगिकी अधिनियम, 2000 - प्रारंभिक https://xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c/e-governance/about-rti-act-2005/93894291a92893e-924915928940915/93894291a92893e-92a94d93094c92694d92f94b91793f91594

C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching content for https://xn--j2bd4cyah0f.xn--11by0av0at5becfj.xn--h2brj9c/e-governance/about-rti-act-2005/93894291a92893e-924915928940915/93894291a92893e-92a94d93094c92694d92f94b91793f915940-90592793f92893f92f92e-2000/93894291a92893e-92a94d93094c92694d92f94b91793f915940-90592793f92893f92f92e-2000-92a94d93093e93090292d93f915
EXCEPTION OCCURRED [Errno 2] No such file or directory: 'VIKASPEDIA_DATA_E_SHASAN\\सूचना-का-अधिकार\\सूचना-तकनीक\\सूचना-प्रौद्योगिकी-अधिनियम,-2000\\सूचना-प्रौद्योगिकी-अधिनियम,-2000---प्रारंभिक\\सूचना-प्रौद्योगिकी-अधिनियम,-2000---प्रारंभिक.txt'


# FLATTEN

In [21]:
import os
import shutil
from pathlib import Path

In [22]:
def move(source,dest):
    isExist = os.path.exists(source)
    os.makedirs(dest, exist_ok=True)
    if isExist is False:
        return "PATH DOES NOT  EXIST"
    try:
        for item in os.listdir(source):
            item_path = os.path.join(source, item)
            if os.path.isdir(item_path):
                move(item_path,dest)  # Recursively call move function for directories
            else:
                # print(item)
                with open(os.path.join(os.getcwd(),item_path),"r",encoding="utf-8") as c:
                    with open(os.path.join(os.getcwd(),dest,clean_name(item)),"w",encoding="utf-8") as nf:
                        nf.write(c.read())
                # print(f"SUCCESS FOR {item}")
    except Exception as e:
        print(e)

In [23]:
source="VIKASPEDIA_DATA_E_SHASAN"
dest="VIKASPEDIA_DATA_E_SHASAN_FLATTENED"
for item in os.listdir(source):
    item_path = os.path.join(source, item)
    if os.path.isdir(item_path):
            move(item_path, os.path.join(dest, item))  # Recursively call move function for directories
             
# move(source,dest)

# INSIGHTS

In [29]:
source="VIKASPEDIA_DATA_E_SHASAN_FLATTENED"
tot=0
for item in os.listdir(source):
    tot=tot+len(os.listdir(os.path.join(source,item)))
    print(item,"--",len(os.listdir(os.path.join(source,item))))
print("total: ",tot)

ई-शासन -- 1
ई-शासन-ऑनलाईन-सेवाएं -- 23
उत्कृष्ट-प्रयास -- 1
एफ.एस.एस.ए.आई(FSSAI)-लाइसेंस-की-आवश्यकता-एवं-प्रक्रिया -- 1
डिजिटल-इंडिया -- 8
डिजिटल-भुगतान -- 19
नागरिक-सेवाएं -- 17
भारत-में-ई-शासन -- 27
भारत-में-विधिक-सेवाएँ -- 26
महिलाएं-एवं-ई-शासन -- 1
मोबाइल-शासन -- 13
राज्य -- 29
वीएलई-के-लिए-संसाधन -- 22
वीएलई-समाचार -- 9
सूचना-का-अधिकार -- 21
total:  218
